#### Cálculo de la similitud entre vectores

In [ ]:
import math 

A = [3.0, 3.0]
B = [1.0,0.5]

# Calcular el producto escalar
producto_escalar = sum(a*b for a, b in zip(A, B))
# Calcular la magnitud de cada vector
magnitud_1 = sum(a*a for a in A)**0.5
magnitud_2 = sum(b*b for b in B)**0.5

# Computar similitud del coseno
similitud_coseno = producto_escalar / (magnitud_1 * magnitud_2)
print(f"Similitud del coseno: {similitud_coseno}")

# Calcular la distancia euclidiana
dist_eucl = math.sqrt((A[0]-B[0])**2 + (A[1]-B[1])**2)
print("Distancia euclidiana: ", dist_eucl)

### Cómputo de la similitud semántica
Textos simples

In [1]:
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain_huggingface import HuggingFaceEmbeddings

data = [
    "En Argentina hay mucha violencia en el futbol.",
    "Las barras bravas del futbol cometen acciones ilegales.",
    "Futbol en inglés se dice soccer.",
    "En el siglo XIX, en Inglaterra, el rugby se denominaba futbol.",
    "Las barras bravas arruinan el espectáculo de futbol.",
    "Los locutores de futbol en radio eran admirables.",
]
docs = [Document(page_content=s) for s in data]
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(docs, embeddings)

query="¿Considera que asistir a la cancha de futbol es peligroso?"
respuestas = vectorstore.similarity_search_with_score(
    query, k=6)

for respuesta in respuestas:
    print(type(respuesta))
    frase = respuesta[0].page_content
    score = respuesta[1]
    print(f"{frase}, {score}")

c:\nuevochatbot\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


<class 'tuple'>
Las barras bravas arruinan el espectáculo de futbol., 0.5986725687980652
<class 'tuple'>
Las barras bravas del futbol cometen acciones ilegales., 0.7578722238540649
<class 'tuple'>
En Argentina hay mucha violencia en el futbol., 0.7673989534378052
<class 'tuple'>
Futbol en inglés se dice soccer., 0.8074974417686462
<class 'tuple'>
Los locutores de futbol en radio eran admirables., 0.8151912689208984
<class 'tuple'>
En el siglo XIX, en Inglaterra, el rugby se denominaba futbol., 0.8350806832313538


#### Búsqueda por similitud semántica en documentos
Propósito del ejemplo  <br>
Aplicar búsqueda por similitud semántica en dos documentos con información diferente.  Se desea verificar que la búsqueda identifica el documento correcto. Se incluye el cálculo del puntaje.


In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
# Carga PDFs
loaders = [
    PyPDFLoader("./directorio/Estructuras.pdf"),
    PyPDFLoader("./directorio/Constitucion1853.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())
# Se aplica RecursiveCharacterTextSplitter. Se define un tamaño de chunk
#     y el solapamiento.
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)
# Se particiona el documento
splits = text_splitter.split_documents(docs)

# Se asigna carpeta donde guardar el vector store para uso posterior
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# Inicializar el espacio vectorial (vector store)
vectordb = Chroma.from_documents(
    documents=splits,  # se le pasa splits
    embedding=embedding, # usa HuggingFaceEmbeddings
)

A partir de aquí inicia la recuperación de los chunks relevantes

In [ ]:
# Hacemos una consulta
consulta = "¿Qué es una lista? "  
#consulta =  "¿Cuales son los derechos? "
# Se buscan documentos similares a la consulta. 
docs = vectordb.similarity_search(consulta, k=3)
print(len(docs))
docs[0].metadata